In [1]:
from itertools import product
import netCDF4 as ncf
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import numpy as np
import math
import csv
from tqdm import tqdm_notebook as tqdm
from keras_tqdm import TQDMNotebookCallback
import os
import random
import tempfile
import pandas as pd
from joblib import dump,load

In [2]:
a=[2,4,7,9,13,17,19]        #a is the hour for which we're trying to predict
b=[0,1,2,3,4,5,6]      # b is indexing for a

#### dates generator. 6 weeks.
no_files=365
nfiles_train=[]
nfiles_test=[]
day_count=180
no_of_months=3
for i in range(no_of_months):
    nfiles_temp=list(range(day_count,day_count+15))  #7
    nfiles_train.append(nfiles_temp)
    nfiles_temp=list(range(day_count+16,day_count+22))  #11
    nfiles_test.append(nfiles_temp)
    day_count=day_count+30
####
##  Grid reference both x-axis,y-axis
loc_y = [165,121,171]
loc_x = [67,58,39]

grid_size_x = dx = 20
grid_size_y = dy = 20

grid_sample_limit_x=[]
grid_sample_limit_y=[]
for i in loc_x:
    grid_sample_limit_x.append((i-15,i+15))
for i in loc_y:
    grid_sample_limit_y.append((i-15,i+15))

In [3]:
grids_x = [np.arange(x-dx, x+dx, 1) for x in loc_x]
grids_y = [np.arange(y-dy, y+dy, 1) for y in loc_y]

all_grids_x = np.concatenate(grids_x)
all_grids_y = np.concatenate(grids_y)

In [4]:
monitors_file = 'C:/Users/woshi/Desktop/CE_675_Project/Data/loc_data/monitor_list_NC.csv'
with open(monitors_file) as csvfile:
    reader = csv.DictReader(csvfile)
    monitor_points = [(int(float(row['Col'])), int(float(row['Row'] ))) for row in reader if row['Col']]

monitors_x, monitors_y = zip(*monitor_points)
all_grids_x=np.concatenate([all_grids_x,monitors_x])
all_grids_y=np.concatenate([all_grids_y,monitors_y])
all_points = list(product(all_grids_x, all_grids_y))


met_dir='C:/Users/woshi/Desktop/CE_675_Project/Data/met_data/'
emis_dir='C:/Users/woshi/Desktop/CE_675_Project/Data/emis_data/'
aq_dir='C:/Users/woshi/Desktop/CE_675_Project/Data/aq_conc/'
short_dir='C:/Users/woshi/Desktop/CE_675_Project/Data/short_data/'

In [5]:
def get_file_list(data_dir):
    return [os.path.join(data_dir, name) for name in os.listdir(data_dir)]


In [6]:
def get_index(grid,val): # get the indices
    try:
        return np.where(grid==val)[0][0]
    except (TypeError,IndexError):
            if(len(val)>1):
                index_list=[]
                for i in val:
                    index_list.append(np.where(grid==i)[0][0])
                return index_list

In [7]:
short_dir='C:/Users/woshi/Desktop/CE_675_Project/Data/short_data/'

short_files = get_file_list(short_dir)

a=[2,4,5,7,9,13,15,17,19,23]

In [8]:
def list_seg(n,look_back,lb_ndarr):
    t_part_list=[]
    i=0
    tlist=lb_ndarr[n]
    def creating_tlist(tlist,j,ind):
        try:
            ind=tlist.index(0-j*24)
            t_part_list.append([i+(0+j*24) for i in tlist[ind:]])
            tlist=tlist[:ind]
            j=j+1
            return creating_tlist(tlist,j,ind)
        except ValueError:
            t_part_list.append([i+(0+j*24) for i in tlist[:ind]])
            return t_part_list
    t_part_list=creating_tlist(tlist,i,0)
    return t_part_list #returns the required sets of indices for a particular hour at request

In [9]:
def hours_list(a,l_b,f_h):
    lb_ndarr = []
    for i in a:
        tlist=list(range(i-l_b,i))           # this is where first_ndarr is created
        lb_ndarr.append(tlist)
    fh_ndarr=[] #this is the list of all indices for the future data collection
    for i in a:
        tlist=list(range(i+1,i+f_h+1))
        fh_ndarr.append(tlist)
    return lb_ndarr,fh_ndarr

In [10]:
def list_seg_future(n,f_h,fh_ndarr):    #if n=2; a[2]=4; tlist=5:29; firstpart=5:23;lastpart=0:5
    t_part_list=[]
    i=1
    tlist=fh_ndarr[n]
    def creating_tlist(tlist,j,ind):
            try:
                ind=tlist.index(0+j*24)
                t_part_list.append([i-(0+j*24) for i in tlist[:ind]])
                tlist=tlist[ind:]
                
                j=j+1
                return creating_tlist(tlist,j,ind)
            except ValueError:
                t_part_list.append([i-(0+j*24) for i in tlist[:24-ind]])
                return t_part_list
    t_part_list=creating_tlist(tlist,i,0)
    return t_part_list

In [11]:
b=[i for i in range(len(a))]      # b is indexing for a

In [12]:
#@jit(nopython=True)
def dist_predictors(x,y, monitor_points):
    """
    Returns distance and angle for an arbitrary point in the domain
    w.r.t. monitor network. These are w.r.t. each monitor or centroid of all monitors
    
    Args:
        x: x_cooridate of arbitrary point
        y: y_coordinate of arbitrary point
        training_params: Training parameters containing locations of monitors
        centroid: if True, returns distance and angle from the centrod of the monitors
    
    Returns:
        Angles and distances 
    
    """
    
    centroid=True
    x_centroid = np.average([point[0] for point in monitor_points])
    y_centroid = np.average([point[1] for point in monitor_points])
    
    def unit_vector(vector):
        """ Returns the unit vector of the vector."""
        return vector / np.linalg.norm(vector)

    def angle_between(v1, v2):
        """ Returns the angle in radians between vectors 'v1' and 'v2'::
                >>> angle_between((1, 0, 0), (0, 1, 0))
                1.5707963267948966
                >>> angle_between((1, 0, 0), (1, 0, 0))
                0.0
                >>> angle_between((1, 0, 0), (-1, 0, 0))
                3.141592653589793
        """
        v1_u = unit_vector(v1)
        v2_u = unit_vector(v2)
        return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

    if centroid:   
        distance = math.sqrt((x_centroid - x)**2 + (y_centroid -y)**2)
        v1 = (x_centroid, y_centroid)
        v2 = (x,y)
        angle = angle_between(v1,v2)
        return np.array([distance, angle])
    else:
        
        distances =[math.sqrt((x - point[0])**2 + (y - point[1])**2) for point in monitor_points]
        angles = [angle_between((x,y), (point[0], point[1])) for point in monitor_points]
        da = distances + angles
        return np.array(da)

In [13]:
#@jit(nopython=True)
def emis_predictors(x,y,i_f_l,tlist):
    d=5
    xs = np.arange(-d,d+1) + x
    ys = np.arange(-d,d+1) + y

    num_cells = len(xs)*len(ys)
    xs_indexed=get_index(all_grids_x,xs)
    ys_indexed=get_index(all_grids_y,ys)
    
    def get_emis_slice(emis_dataset,species,tlist):
            grid = np.ix_(tlist,xs_indexed,ys_indexed)
            e1 = emis_dataset[species][:]
            e2 = e1[grid]
            return e2.reshape(len(tlist), num_cells)
    for i in range(len(i_f_l)):
      emis_dataset=train_param(i_f_l[i])['emis_dataset']
      emis_species=train_param(i_f_l[i])['emis_species']
      if(i==0):
        emis_seq = [ get_emis_slice(emis_dataset, species,tlist[i]) for species in emis_species] 
        emis_seq = np.concatenate(emis_seq,axis=1)
        emis_seq_total=emis_seq
      else:
          emis_seq = [ get_emis_slice(emis_dataset, species,tlist[i]) for species in emis_species] 
          emis_seq = np.concatenate(emis_seq,axis=1)
          emis_seq_total=np.vstack((emis_seq_total,emis_seq))
    return emis_seq_total

In [14]:
def history_monitors_predictors(t,i_f_l, monitor_points):

    monitors_x, monitors_y = zip(*monitor_points) 

    monitors_x_edit=get_index(all_grids_x,monitors_x)
    monitors_y_edit=get_index(all_grids_y,monitors_y)

    for i in range(len(i_f_l)):
        aq_input_data=train_param(i_f_l[i])['aq_input_data']  
        if(i==0):
          concs = aq_input_data['O3'][t[i]][:,monitors_x_edit,monitors_y_edit]
          concs_total=concs
        else:
          concs = aq_input_data['O3'][t[i]][:,monitors_x_edit,monitors_y_edit]
          concs_total=np.vstack((concs_total,concs))  
    return concs_total

In [15]:
def met_predictors(t,i_f_l,monitor_points):
   
    monitors_x, monitors_y = zip(*monitor_points)     
    monitors_x_edit=get_index(all_grids_x,monitors_x)    #function to get index
    monitors_y_edit=get_index(all_grids_y,monitors_y)    #function to get index
    
    def get_met(met_dataset,met_params,t):
      met_seq = []
      for param in met_params:
          m1 = met_dataset[param][:]
          m2 = m1[t][:,monitors_x_edit,monitors_y_edit]
          met_seq.append(m2)
      return met_seq
    for i in range(len(i_f_l)):
        met_dataset=train_param(i_f_l[i])['met_dataset']
        met_params=train_param(i_f_l[i])['met_params']
        if(i==0):
            met_seq = get_met(met_dataset,met_params,t[i])
            met_seq=np.concatenate(met_seq, axis=1)
            met_seq_total=met_seq
        else:
            met_seq = get_met(met_dataset,met_params,t[i])
            met_seq=np.concatenate(met_seq, axis=1)
            met_seq_total=np.vstack((met_seq_total,met_seq))
    return met_seq_total

In [16]:
def get_predictors_xyt(p,tlist,i_f_l,debug=False):
    """
    For each point and timestamp(s) generate a row of predictors
    """
    
    x=p[0]
    y=p[1]
    dist = dist_predictors(x,y,monitor_points) #time invariant
    dist_tile = np.tile(dist,(sum([len(i) for i in tlist]),1))

    
    emis = emis_predictors(x,y,i_f_l,tlist)
    met  = met_predictors(tlist,i_f_l,monitor_points)
    
    hist = history_monitors_predictors(tlist,i_f_l,monitor_points)
   
    try:
        preds = np.concatenate([dist_tile, emis, hist,met], axis=1)
        return preds
    except:
        return [dist_tile,emis,hist,met]

In [17]:
def get_aq_input(x,y,t,i_f_l):
    x_indexed=get_index(all_grids_x,x)
    y_indexed=get_index(all_grids_y,y)
    
    for i in range(len(i_f_l)):
        #print(len(i_f_l))
        #print(len(t))
        
        aq_input_data=train_param(i_f_l[i])['aq_input_data']  
        if(i==0):
          aq_input = aq_input_data['O3'][t[i]][:,x_indexed,y_indexed]
          aq_input_total=aq_input
        else:
          aq_input = aq_input_data['O3'][t[i]][:,x_indexed,y_indexed]
          aq_input_total=np.concatenate((aq_input_total,aq_input))  
    return aq_input_total

In [18]:
def get_future_data(o_f_l,tlist,point_x,point_y):
    for i in range(len(o_f_l)):
      aq_output_data=train_param(o_f_l[i])['o3_dataset']
      if(i==0):
        aq_output=aq_output_data['O3'][tlist[i],point_x,point_y]
        aq_output_total=aq_output
        #print(len(aq_output))
      else:
        aq_output=aq_output_data['O3'][tlist[i],point_x,point_y]
        print(aq_output_total)
        print(aq_output)
        aq_output_total=np.concatenate((aq_output_total,aq_output))
    
    return aq_output_total

In [19]:
def train_param(file_num):
    infile=np.load(short_dir+str(file_num)+'.npz')
    

        #print(i)
    
    #outfile=np.load(short_dir+str(out_file_list[0])+'.npz')
    #outfile=outfile['O3']
    #f_output=tempfile.TemporaryFile()
    #for i in range(1,len(out_file_list)):
    #    file_1=np.load(short_dir+str(out_file_list[i])+'.npz')
    #    np.savez(f_output,O3=create_npz(file_1,outfile,'O3'))
    #    _ = f_output.seek(0) # Only needed here to simulate closing & reopening file
    #    outfile=np.load(f_output)
    #f_output = tempfile.NamedTemporaryFile(delete=True)
    #f_output.close()
    training_params = {
                   'd' : 5, #local_emissions_size
                   'emis_species': ['NO', 'NO2'], #emission_species
                   'met_params':['PBL', 'Q2', 'TEMP2', 'WSPD10', 'WDIR10'], #met parameters
                   #'met_params': ['PBL', 'WDIR10'],
                   'emis_dataset': infile,
                   'met_dataset':infile,
                   'aq_input_data':infile,
                   'o3_dataset': infile,
                   'monitor_points':monitor_points
                 }
    return training_params

In [20]:
def limiting_points(cell_list,grid_sample_limit):
    return cell_list[np.logical_or(np.logical_and((cell_list<grid_sample_limit[0][1]),
    (cell_list>grid_sample_limit[0][0])),np.logical_and((cell_list<grid_sample_limit[1][1]),
    (cell_list>grid_sample_limit[1][0])),np.logical_and((cell_list<grid_sample_limit[2][1]),
                                                        (cell_list>grid_sample_limit[2][0])))]

In [21]:
def sampling_function(all_points,x_no,y_no):
    x_cells=[]
    y_cells=[]
    
    x_cells=random.sample(list(all_grids_x),k=x_no)
    y_cells=random.sample(list(all_grids_y),k=y_no)  
    x_limited=limiting_points(np.array(x_cells),grid_sample_limit_x)
    y_limited=limiting_points(np.array(y_cells),grid_sample_limit_y)

    x_y=list(set(product(x_limited,y_limited)))
    return x_y

In [22]:
def tqdm_function(x_y_t_f,files,a,look_back,f_h):
      predictors={}
      aq_data={}
      aq_input={}
      def round_func(val):
        if(val%1>0):
          return int(val)+1
        else:
          return int(val)
      l_edge=files[round_func(look_back/24) -1]
      r_edge=files[-1*int(f_h/24)]
      lb_ndarr,fh_ndarr=hours_list(a,look_back,f_h)
      for i in x_y_t_f:
          if((i[0]>l_edge) and (i[0]<r_edge)):
              #inp_file_list = range(i[0]-round_func(look_back/24),i[0]+1)
              #out_file_list=range(i[0],i[0]+int(f_h/24)+1)
              #o3_dataset=train_param(out_file_list)['o3_dataset']


              point=i[1][0]#[0]                 ## x y coordinates of the current point


              #hour_next = a[i[1][1]]+1  #hour to be forecasted.
              point_x=get_index(all_grids_x,int(point[0]))
              point_y=get_index(all_grids_y,int(point[1]))
              #lb_ndarr,fh_ndarr=hours_list(look_back,f_h)
              #print(list(inp_file_list))
              #print(list(out_file_list))
              #o3_dataset=o3_dataset['O3']
              hours_input=list_seg(i[1][1],look_back,lb_ndarr)
              hours_output=list_seg_future(i[1][1],f_h,fh_ndarr)
              #print(type(o3_dataset[fh_ndarr[i[1][1]],point_x,point_y]))
              inp_file_list=range(i[0]-len(hours_input),i[0])#+1)
              out_file_list=range(i[0],i[0]+len(hours_output))
              aq_data[i]=get_future_data(out_file_list,hours_output,point_x,point_y)
              print(len(aq_data[i]))
              #print(aq_data[i])
              aq_input[i]= get_aq_input(point[0],point[1],hours_input,inp_file_list)
              print(i[0])
              #print(type(get_predictors_xyt(point,lb_ndarr[i[1][1]],training_params,debug=False)))
              predictors[i]=get_predictors_xyt(point,hours_input,inp_file_list,debug=False)
              #print(len(predictors[i][1]))
              #print(predictors[i][1])
              #print(aq_data[i])
      return predictors, aq_data, aq_input

In [23]:
def feat_ret(nfiles,a,n1,n2,look_back,f_h):
    aq_data={}

    predictors={}
    aq_input={}
    x_y=[]
    x_y=sampling_function(all_points,n1,n2)
    x_y.append(tuple([58,121]))
    x_y_t=product(x_y,b)
    x_y_t=list(x_y_t)
    
    for f_no in range(len(nfiles)):
        x_y_t_f=product(nfiles[f_no],x_y_t) 

        x_y_t_f=list(x_y_t_f)
        print(len(x_y_t_f))
        print(f_no)
        predictors_temp,aq_data_temp,aq_input_temp=tqdm_function(x_y_t_f,nfiles[f_no],a,look_back,f_h)
        predictors.update(predictors_temp)
        aq_data.update(aq_data_temp)
        aq_input.update(aq_input_temp)

    return predictors,aq_data,aq_input,x_y

In [24]:
look_back=30
f_h=24
#a=[2,4,5,7,9,13,15,17,19,22] #the same list of hours is repeated for all the days
predictors,aq_data,aq_input,x_y=feat_ret(nfiles_train,a,25,15,look_back,f_h)
# print ('\n Done getting predictors')
# In[ ]:

predictors_future,aq_data_future,aq_input_future,x_y_future=feat_ret(nfiles_train,a,15,12,look_back,f_h)

24450
0


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/woshi/Desktop/CE_675_Project/Data/short_data/182.npz'

In [29]:
save_to_file = True
read_from_file = False
analyze = True
create_plots = True


# In[14]:



def get_splits(predictors,aq_data,train_fraction=0.95):
    num_predictors = len(predictors)
    pxy = list(predictors.keys())
    #print(len(aq_data))
    num_train = int(train_fraction*num_predictors)
    train_indices = np.random.choice(np.arange(num_predictors),num_train, replace=False)
    pxy_train = [pxy[i] for i in train_indices]
    predictors_train = [predictors[i] for i in pxy_train]
    aq_train=[]
    for i in pxy_train:
        aq_train.append(aq_data[i])
    #aq_train = [aq_data[i] for i in pxy_train]
    print ('number of training points', num_train, '&', len(predictors_train))

    #Random sample for Testing
    predictors_test = [predictors[i] for i in pxy if i not in pxy_train]
    aq_test = [aq_data[i] for i in pxy if i not in pxy_train]
    pxy_test = [i for i in pxy if i not in pxy_train]
    print('number of testing points', len(predictors_test), '&', len(aq_test))
    return predictors_train, aq_train, predictors_test, aq_test

if not read_from_file:
    print('Getting train and test splits')
    predictors_train, aq_train, predictors_test, aq_test = get_splits(predictors,aq_data)
    predictors_train_future,aq_train_future,predictors_test_future = get_splits(predictors_future,aq_data_future)

Getting train and test splits


NameError: name 'predictors' is not defined

In [30]:
# ### 6. Data Standerdization

# In[15]:


type(predictors_train)
type(predictors_train_future)

NameError: name 'predictors_train' is not defined

In [31]:
# In[16]:


def scaling_data(predictors,aq_data):
    if not read_from_file:
    
        print('scaling inputs and outputs')
        from sklearn.preprocessing import MinMaxScaler
    
        data = np.concatenate(list(predictors.values()))
        print(len(data[1]))
        pred_scaler = MinMaxScaler()
        pred_scaler.fit(data)
    
        print(len(aq_data))
        for i in aq_data:
          print(len(aq_data[i]))
        aq_data_1 = np.stack(aq_data.values()).reshape(-1,24)
        aq_scaler = MinMaxScaler()
        aq_scaler.fit(aq_data_1)
    
        predictors_train_scaled = [pred_scaler.transform(mx) for mx in predictors_train]
        predictors_train_rnn = np.stack(predictors_train_scaled)
        aq_train_rnn = aq_scaler.transform(np.stack(aq_train).reshape(-1,24))
    
        predictors_test_scaled = [pred_scaler.transform(mx) for mx in predictors_test]
        predictors_test_rnn = np.stack(predictors_test_scaled)
        aq_test_rnn = aq_scaler.transform(np.stack(aq_test).reshape(-1,24))
    return predictors_train_rnn,predictors_test_rnn,aq_train_rnn,aq_test_rnn,pred_scaler,aq_scaler
output_dir='/mnt/raid2/System/home/mmohan3/weather_data/output_data/'

In [32]:
### Save for future use.
predictors_file = '/mnt/raid2/System/home/mmohan3/predictors_data3_poster.npz'
predictors_future_file = '/mnt/raid2/System/home/mmohan3/predictors_data3_future.npz'
if save_to_file == True:
    #data_to_save = [predictors_train_rnn, aq_train, predictors_test_rnn,aq_test] 
    predictors_train_rnn,predictors_test_rnn,aq_train_rnn,aq_test_rnn,pred_scaler,aq_scaler=scaling_data(predictors,aq_data)
    np.savez(predictors_file, predictors_train_rnn = predictors_train_rnn, #X_train, shape=(samples,tsteps,features)
                              aq_train_rnn = aq_train_rnn, #y_train
                              predictors_test_rnn = predictors_test_rnn, #X_test
                              aq_test_rnn = aq_test_rnn) #y_test)
    dump(pred_scaler,output_dir+'pred_scaler.joblib')
    dump(aq_scaler,output_dir+'aq_scaler.joblib')
    predictors_train_future_rnn,predictors_test_future_rnn,aq_train_future_rnn,aq_test_future_rnn,pred_future_scaler,aq_future_scaler=scaling_data(predictors_future,aq_data)
    np.savez(predictors_future_file, predictors_train_future_rnn = predictors_train_future_rnn, #X_train, shape=(samples,tsteps,features)
                              aq_train_future_rnn = aq_train_future_rnn, #y_train
                              predictors_test_future_rnn = predictors_test_future_rnn, #X_test
                              aq_test_future_rnn = aq_test_future_rnn) #y_test        
    dump(pred_scaler,output_dir+'pred_future_scaler.joblib')
    dump(aq_scaler,output_dir+'aq_future_scaler.joblib')

NameError: name 'predictors' is not defined